In [1]:
import tensorflow as tf
import numpy as np

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Dense,
    Conv2D,
    Flatten,
    ReLU,
    Softmax,
    AveragePooling2D,
    BatchNormalization
    )

import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc

import hls4ml


2022-03-16 13:16:29.357015: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-10.1/lib64:/opt/rh/devtoolset-9/root/usr/lib64:/opt/rh/devtoolset-9/root/usr/lib:/opt/rh/devtoolset-9/root/usr/lib64/dyninst:/opt/rh/devtoolset-9/root/usr/lib/dyninst:/opt/rh/devtoolset-9/root/usr/lib64:/opt/rh/devtoolset-9/root/usr/lib:/usr/local/cuda-10.1/lib64:
2022-03-16 13:16:29.357116: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


WARN: Unable to import optimizer(s) from qkeras.py: No module named 'qkeras'


/afs/cern.ch/work/e/egovorko/miniconda3/envs/tf2.7/lib/python3.9/site-packages/hls4ml/converters/__init__.py:15: UserWarning: WARNING: Pytorch converter is not enabled!
  warnings.warn("WARNING: Pytorch converter is not enabled!")


In [2]:
wrapped = 1


In [3]:
def hls4ml_model(input_shape):
    from hls4ml.wrappers import keras as K
    # Creating a Sequential Model and adding the layers
    model = K.Sequential(default_precision='ap_fixed<16,6>', output_dir='hls4mlprj_train')
    ##
    model.add(K.layers.Conv2D(28, kernel_size=(3,3), input_shape=input_shape))
    model.add(K.layers.AveragePooling2D(pool_size=(2, 2)))
    model.add(K.layers.Conv2D(28, kernel_size=(3,3)))
    model.add(K.layers.AveragePooling2D(pool_size=(2, 2)))
    model.add(K.layers.Flatten()) # Flattening the 2D arrays for fully connected layers
    model.add(K.layers.Dense(54))
    model.add(K.layers.ReLU())
    model.add(K.layers.BatchNormalization())
    model.add(K.layers.Dense(10))
    model.add(K.layers.Softmax(skip_wrapping=True))
    return model

In [4]:

def keras_model(input_shape):
    # Creating a Sequential Model and adding the layers
    model = Sequential()
    model.add(Conv2D(28, kernel_size=(3,3), input_shape=input_shape))
    model.add(AveragePooling2D(pool_size=(2, 2)))
    model.add(Conv2D(28, kernel_size=(3,3)))
    model.add(AveragePooling2D(pool_size=(2, 2)))
    model.add(Flatten()) # Flattening the 2D arrays for fully connected layers
    model.add(Dense(54))
    model.add(ReLU())
    model.add(BatchNormalization())
    model.add(Dense(10))
    model.add(Softmax())
    model.summary()
    return model


In [5]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()


In [6]:
# Reshaping the array to 4-dims so that it can work with the Keras API
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
input_shape = (28, 28, 1)
# Making sure that the values are float so that we can get decimal points after division
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
# Normalizing the RGB codes by dividing it to the max RGB value.
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print('Number of images in x_train', x_train.shape[0])
print('Number of images in x_test', x_test.shape[0])


x_train shape: (60000, 28, 28, 1)
Number of images in x_train 60000
Number of images in x_test 10000


In [7]:
# convert class vectors to binary class matrices
num_classes = 10
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

In [ ]:
model = hls4ml_model(input_shape) if wrapped else keras_model(input_shape)
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(x=x_train, y=y_train, epochs=5)


2022-03-16 13:16:34.365530: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-16 13:16:34.366108: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-16 13:16:34.366611: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-10.1/lib64:/opt/rh/devtoolset-9/root/usr/lib64:/opt/rh/devtoolset-9/root/usr/lib:/opt/rh/devtoolset-9/root/usr/lib64/dyninst:/opt/rh/devtoolset-9/root/usr/lib/dyninst:/opt/rh/devtoolset-9/root/usr/lib64:/opt/rh/devtoolset-9/root/usr/lib:/usr/local/cuda-10.1/lib64:
2022-03-16 13:16:34.366695: W t

Epoch 1/5
 341/1875 [====>.........................] - ETA: 5:11 - loss: 0.9610 - accuracy: 0.7478

In [ ]:
pred = model.predict(x_test)


In [ ]:
if wrapped:
    hls_model = hls4ml.converters.convert_from_wrapped_model(
        model,
        output_dir='output/')
else:
    config = hls4ml.utils.config_from_keras_model(
        model,
        granularity='name',
        default_precision='ap_fixed<8,4>')
    hls_model = hls4ml.converters.convert_from_keras_model(
        model,
        hls_config=config,
        output_dir='output/')